In [25]:
import requests
import pandas as pd
import numpy as np



In [47]:
manifesto_data = pd.read_csv("manifesto_preprocessed.csv")
manifesto_data.head()

,libertad_personal_score,libertad_economica_score,text_processed
0,0,0,declaración principiosfrente victorio
1,0,0,argentino convicción capacidad construir nuevo...
2,0,0,crisis ser rostro país ultima década
3,0,0,crisis económico social institucional político...
4,1,-1,regla juego claro justo económico social único...


In [48]:
# Ver distribución de los puntajes
print("Distribución de libertad_personal_score:")
print(manifesto_data['libertad_personal_score'].value_counts())

print("\nDistribución de libertad_economica_score:")
print(manifesto_data['libertad_economica_score'].value_counts())

Distribución de libertad_personal_score:
libertad_personal_score
 0    346331
-1     58982
 1     42784
Name: count, dtype: int64

Distribución de libertad_economica_score:
libertad_economica_score
 0    272578
-1     94401
 1     81118
Name: count, dtype: int64


In [49]:
# Ver combinaciones de ambas columnas
combinaciones = manifesto_data.groupby(['libertad_personal_score', 'libertad_economica_score']).size().reset_index(name='counts')
print("\nCombinaciones de puntajes:")
print(combinaciones)


Combinaciones de puntajes:
   libertad_personal_score  libertad_economica_score  counts
0                       -1                        -1   41087
1                       -1                         0   17895
2                        0                        -1   28396
3                        0                         0  236817
4                        0                         1   81118
5                        1                        -1   24918
6                        1                         0   17866


In [74]:
import pandas as pd
from sklearn.utils import resample

# Paso 1: Subsampling de las clases (0, 0) y otras combinaciones con ceros
manifesto_0_0 = manifesto_data[(manifesto_data['libertad_personal_score'] == 0) & (manifesto_data['libertad_economica_score'] == 0)]
manifesto_0_menos1 = manifesto_data[(manifesto_data['libertad_personal_score'] == 0) & (manifesto_data['libertad_economica_score'] == -1)]
manifesto_0_1 = manifesto_data[(manifesto_data['libertad_personal_score'] == 0) & (manifesto_data['libertad_economica_score'] == 1)]
manifesto_otros = manifesto_data[(manifesto_data['libertad_personal_score'] != 0) | (manifesto_data['libertad_economica_score'] != 0)]

# Reducir el tamaño de la clase (0, 0) y otras clases con ceros
manifesto_0_0_reducido = manifesto_0_0.sample(frac=0.05, random_state=42)  # Submuestreo al 10%
manifesto_0_menos1_reducido = manifesto_0_menos1.sample(frac=0.1, random_state=42)  # Submuestreo al 10%
manifesto_0_1_reducido = manifesto_0_1.sample(frac=0.1, random_state=42)  # Submuestreo al 10%

# Recombinar el dataset con el subsampling aplicado
manifesto_balanceado = pd.concat([manifesto_otros, manifesto_0_0_reducido, manifesto_0_menos1_reducido, manifesto_0_1_reducido])

# Paso 2: Upsampling de las clases subrepresentadas (-1 y 1)

# Separar las clases subrepresentadas
manifesto_menos1 = manifesto_balanceado[manifesto_balanceado['libertad_personal_score'] == -1]
manifesto_1 = manifesto_balanceado[manifesto_balanceado['libertad_personal_score'] == 1]

# Aumentar en un 20% de su tamaño original
n_menos1 = len(manifesto_menos1)
n_1 = len(manifesto_1)

# Upsampling de las clases -1 y 1 con un incremento del 20%
manifesto_menos1_upsampled = resample(manifesto_menos1,
                                      replace=True,  # Permitir replicación de datos
                                      n_samples=int(n_menos1 * 1.2),  # Aumentar en 20%
                                      random_state=42)

manifesto_1_upsampled = resample(manifesto_1,
                                 replace=True,
                                 n_samples=int(n_1 * 1.5),  # Aumentar en 20%
                                 random_state=42)

# Recombinar el dataset con el upsampling aplicado
manifesto_balanceado_upsampled = pd.concat([manifesto_balanceado[manifesto_balanceado['libertad_personal_score'] == 0],
                                            manifesto_menos1_upsampled,
                                            manifesto_1_upsampled])


# Verificar el nuevo tamaño del dataset
print("Tamaño original del dataset:", len(manifesto_data))
print("Tamaño del dataset balanceado:", len(manifesto_balanceado_upsampled))

Tamaño original del dataset: 448097
Tamaño del dataset balanceado: 267261


In [75]:
# Verificar la nueva distribución de las combinaciones
combinaciones_finales = manifesto_balanceado_upsampled.groupby(['libertad_personal_score', 'libertad_economica_score']).size().reset_index(name='counts')
print("\nCombinaciones después del balanceo:")
print(combinaciones_finales)

# Verificar la distribución de libertad_personal_score
print("\nDistribución de libertad_personal_score después del balanceo:")
print(manifesto_balanceado_upsampled['libertad_personal_score'].value_counts())

# Verificar la distribución de libertad_economica_score
print("\nDistribución de libertad_economica_score después del balanceo:")
print(manifesto_balanceado_upsampled['libertad_economica_score'].value_counts())


Combinaciones después del balanceo:
   libertad_personal_score  libertad_economica_score  counts
0                       -1                        -1   49207
1                       -1                         0   21571
2                        0                        -1   31236
3                        0                         0   11841
4                        0                         1   89230
5                        1                        -1   37612
6                        1                         0   26564

Distribución de libertad_personal_score después del balanceo:
libertad_personal_score
 0    132307
-1     70778
 1     64176
Name: count, dtype: int64

Distribución de libertad_economica_score después del balanceo:
libertad_economica_score
-1    118055
 1     89230
 0     59976
Name: count, dtype: int64


In [77]:
manifesto_balanceado_upsampled.to_csv("manifesto_balanceado.csv", index=False)